In [ ]:
from magnet.base import Magnet

config = {
    "host": "localhost",
    "credentials": None,
    "domain": None,
    "stream_name": "my_stream",
    "category": "my_category",
    "kv_name": "my_kv",
    "session": "my_session",
    "os_name": "my_object_store",
    "index": {
        "milvus_uri": "127.0.0.1",
        "milvus_port": 19530,
        "milvus_user": "test",
        "milvus_password": "test",
        "dimension": 1024,
        "model": "BAAI/bge-large-en-v1.5",
        "name": "test",
        "options": {
            'metric_type': 'COSINE',
            'index_type':'HNSW',
            'params': {
                "efConstruction": 40
                , "M": 48
            }
        }
    }
}

magnet = Magnet(config)
await magnet.align()

In [ ]:
from magnet.ic.field import Resonator, Charge
from magnet.utils.data_classes import Payload

from tractor_beam.utils.file_handlers import PDFProcessor

import json

reso, field = Resonator(magnet), Charge(magnet)
processor = PDFProcessor()
models = processor.load_models()

await reso.on(obj=True)

async def handle_jobs(payload, msg):
    msg['_isClaimed'] = True
    await magnet.kv.update(msg['_id'], json.dumps(msg).encode('utf-8'), payload.revision)
    objects = await magnet.os.list()
    for obj in objects:
        if obj.name == msg['_id']:
            await reso.download(obj)
            file = await processor.export_to_markdown(f"./{obj.name}.{obj.headers['ext']}", f"./", obj.name, models)
            with open(f"{file}/{file.split('/')[-1]}.md", "r") as md:
                content = md.read()
                await field.pulse(Payload(content, file.split('/')[-1]), v=True)

worker = await reso.worker(cb=handle_jobs)